# REPORTE OPPM-Kairos

Para la generación del siguiente informe se realizaron las siguientes actividades, detalladas a continuación:

1. **Generación del Algoritmo de búsqueda GTIN**: este algoritmo nos permite identificar los códigos prioritarios y los atributos que tengan similutud a los datos del observatorio, por ejemplo: **Registro Sanitario, fracciones, etc**. Para dicho proceso se utilizó el **Catalogo.GTIN** provisto por SuSalud.El algoritmo está conformado por dos metodologías de búsqueda, la primera basada en búsqueda mediante un modelo de **machine learning** (Freetextable) y la segunda basada en **coincidencias** en el contenido de palabras (Containstable). El algoritmo tiene como objetivo comprar ambos catálogos (GTIN y Kairos) para dar como resultado las dos mejores posibilidades de coincidencia.

In [ ]:
--CATALOGO DE GTIN PROVISTO POR SUSALUD
SELECT *
FROM Catalogo.GTIN CG

In [ ]:
--CATALOGO GTIN DE PRODUCTOS DECLARADOS EN EL TEDEF (6926 registros)
SELECT *
FROM Catalogo.Consumo_TEDEF CT

In [ ]:
--RELACIÓN DE CÓDIGOS GTIN PARA IDENTIFICAR LOS ATRIBUTOS DEL PRODUCTO QUE PUEDAN CORRESPONDER CON EL CATALOGO DE MEDICAMENTOS DEL OPPM
SELECT CG.CODIGO, CG.NUMREGISTROSANITARIO, CG.DENOMINACIONCOMUN, CG.NOMBRE, CG.CONCENTRACION, CG.UNIDADENVASE, CG.FORMAFARMACEUTICASIMP, CG.LABORATORIO
FROM Catalogo.GTIN CG
INNER JOIN Catalogo.Consumo_TEDEF CT
ON CG.Codigo = CT.GTIN

In [ ]:
--ALGORITMO DE BÚSQUEDA GTIN: BUSCA Y ALMACENA LA INFORMACIÓN EN UNA TABLA QUE SERÁ UTILIZADA POSTERIORMENTE
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE PROCEDURE [dbo].[sp_OPPM_GTIN_V2] @INICIO INT
AS
DECLARE @Codigo         NVARCHAR(30),
        @RegSan         NVARCHAR(30),
        @Prin_Activo    NVARCHAR(255),
        @Nombre         NVARCHAR(255),
        @Lab            NVARCHAR(255),
        @Concent        NVARCHAR(255),
        @Fraccion       NVARCHAR(255),
        @Present        NVARCHAR(255),
        @Form_Farm      NVARCHAR(255),
        @Prin_Activo_s  NVARCHAR(255),
        @Nombre_s       NVARCHAR(255),
        @Lab_s          NVARCHAR(255),
        @Concent_s      NVARCHAR(255),
        @Fraccion_s     NVARCHAR(255),
        @Form_Farm_s    NVARCHAR(255),
        @GTIN           NVARCHAR(255),
        @PK             INT = @INICIO,
        @MAX            INT,
        @y              NVARCHAR(1500),
        @x              NVARCHAR(1500),
        @PESO1          NVARCHAR(25),
        @PESO2          NVARCHAR(25),
        @PESO3          NVARCHAR(25),
        @PESO4          NVARCHAR(25)


SET @MAX = (SELECT MAX(pk_Catalogo_producto) FROM Productos)
SET @PESO1 = ' WEIGHT(0.6), '
SET @PESO2 = ' WEIGHT(1), '

WHILE @PK <= @MAX
BEGIN

SET @Prin_Activo_s = (SELECT UPPER(Prin_Activo) FROM Productos WHERE PK_Catalogo_Producto = @PK)
SET @Nombre_s = (SELECT UPPER(Nom_Prod) FROM Productos WHERE PK_Catalogo_Producto = @PK)
SET @Lab_s = (SELECT UPPER(Laboratory) FROM Productos INNER JOIN Productos_Laboratorio ON Productos.Cod_Prod = Productos_Laboratorio.Cod_Prod WHERE PK_Catalogo_Producto = @PK)
SET @Concent_s = (SELECT Concent FROM Productos WHERE PK_Catalogo_Producto = @PK)
SET @Fraccion_s = (SELECT convert(NVARCHAR(50),Fracciones) FROM Productos WHERE PK_Catalogo_Producto = @PK)
SET @Form_Farm_s = (SELECT UPPER(P.Nom_Form_Farm_Simplif) FROM Productos AS P LEFT JOIN MAESTRO_FORMA_FARM AS M ON p.Nom_Form_Farm_Simplif = M.Nom_Form_Farm_Simplif WHERE PK_Catalogo_Producto = @PK)
SET @Codigo = (SELECT Cod_Prod FROM Productos WHERE PK_Catalogo_Producto = @PK)
SET @Present = (SELECT Presentac FROM Productos WHERE PK_Catalogo_Producto = @PK)
SET @GTIN = (SELECT Cod_GTIN FROM Productos WHERE PK_Catalogo_Producto = @PK)
SET @RegSan = (SELECT Num_RegSan FROM Productos WHERE PK_Catalogo_Producto = @PK)

SET @Prin_Activo = (CASE WHEN ISNULL(@Prin_Activo_s,'') = '' THEN '' ELSE ISNULL(@Prin_Activo_s,'') + ' ' END)
SET @Nombre = (CASE WHEN ISNULL(@Nombre_s,'') = '' THEN '' ELSE ISNULL(@Nombre_s,'') + ' ' END)
SET @Lab = (CASE WHEN ISNULL(@Lab_s,'') = '' THEN '' ELSE ISNULL(@Lab_s,'') + ' ' END)
SET @Concent = (CASE WHEN ISNULL(@Concent_s,'') = '' THEN '' ELSE ISNULL(@Concent_s,'') + ' ' END)
SET @Fraccion = (CASE WHEN ISNULL(@Fraccion_s,'') = '' THEN '' ELSE ISNULL(@Fraccion_s,'') + ' ' END)
SET @Form_Farm = (CASE WHEN ISNULL(@Form_Farm_s,'') = '' THEN '' ELSE ISNULL(@Form_Farm_s,'') + ' ' END)

--SET @x = @Prin_Activo + @Nombre + @Lab + @Concent + @Fraccion + REPLACE(@Form_Farm,' ','');
SET @x = @RegSan

SET @Prin_Activo = (CASE WHEN ISNULL(@Prin_Activo_s,'') = '' THEN '' ELSE '"'+ ISNULL(@Prin_Activo_s,'') + '"' + @PESO1 END)
SET @Nombre = (CASE WHEN ISNULL(@Nombre_s,'') = '' THEN '' ELSE '"' + ISNULL(@Nombre_s,'') + '"' + @PESO2 END)
SET @Lab = (CASE WHEN ISNULL(@Lab_s,'') = '' THEN '' ELSE '"' + ISNULL(@Lab_s,'') + '"' + @PESO2 END)
SET @Concent = (CASE WHEN ISNULL(@Concent_s,'') = '' THEN '' ELSE '"' + ISNULL(@Concent_s,'') + '"' + @PESO1 END)
SET @Fraccion = (CASE WHEN ISNULL(@Fraccion_s,'') = '' THEN '' ELSE '"' + ISNULL(@Fraccion_s,'') + '"' + @PESO2 END)
SET @Form_Farm = (CASE WHEN ISNULL(@Form_Farm_s,'') = '' THEN '' ELSE '"' + ISNULL(@Form_Farm_s,'') + '"' + @PESO1 END)

--SET @y = 'ISABOUT(' + @Prin_Activo + @Nombre + @Lab + @Concent + @Fraccion + REPLACE(@Form_Farm,', ','') + ')';
SET @y = 'ISABOUT(' + '"' + @RegSan + '"' + REPLACE(@PESO2,', ','') + ')';
WITH x AS
(
    SELECT @PK [PK], KEY_TBL.[RANK], @Codigo [OPPM_CODIGO], [Código] [GTIN], @GTIN [OPPM_GTIN], Nro_Registro_Sanitario [RegSan], @RegSan [OPPM_RegSan], Nombre, REPLACE(@Nombre_s,', ','') [OPPM_NOM], Principio_Activo, REPLACE(@Prin_Activo_s,', ','') [OPPM_PRIN_ACT], Laboratorio, REPLACE(@Lab_s,', ','') [OPPM_LAB], Concentración, REPLACE(@Concent_s,', ','') [OPPM_CONCENT], Unidad_Envase [Fraccion], REPLACE(@Fraccion_s,', ','') [OPPM_FRAC], Forma_Farmacéutica_Simple, REPLACE(@Form_Farm_s,', ','') [Forma_Farm], Presentación, @Present [OPPM_PRES], 'F' [TB]
    FROM GTIN_13
    INNER JOIN FREETEXTTABLE(GTIN_13, Nro_Registro_Sanitario,
                                        @x, 2) AS KEY_TBL
                    ON [pk_GTIN_13] = KEY_TBL.[KEY]
), y AS
(
    SELECT @PK [PK], KEY_TBL.[RANK], @Codigo [OPPM_CODIGO], [Código] [GTIN], @GTIN [OPPM_GTIN], Nro_Registro_Sanitario [RegSan], @RegSan [OPPM_RegSan], Nombre, REPLACE(@Nombre_s,', ','') [OPPM_NOM], Principio_Activo, REPLACE(@Prin_Activo_s,', ','') [OPPM_PRIN_ACT], Laboratorio, REPLACE(@Lab_s,', ','') [OPPM_LAB], Concentración, REPLACE(@Concent_s,', ','') [OPPM_CONCENT], Unidad_Envase [Fraccion], REPLACE(@Fraccion_s,', ','') [OPPM_FRAC], Forma_Farmacéutica_Simple, REPLACE(@Form_Farm_s,', ','') [Forma_Farm], Presentación, @Present [OPPM_PRES], 'C' [TB]
    FROM GTIN_13
    INNER JOIN CONTAINSTABLE(GTIN_13, Nro_Registro_Sanitario,
                                        @y, 2) AS KEY_TBL
                    ON [pk_GTIN_13] = KEY_TBL.[KEY]
), xy AS

(
    SELECT *
    FROM x
    UNION ALL
    SELECT *
    FROM y
)
INSERT INTO OPPM_GTIN
SELECT TOP 2 *
FROM xy
ORDER BY [RANK] DESC

SET @PK = @PK + 1

END

GO


2. **Revisión por pares en 3 grupos**: Una vez ejecutado el algoritmo, se obtiene como resultado una lista de **24 mil registros** a comprar, con **top 2** de medicamentos con mayores coincidencias para cada producto del catálogo del observatorio. Para poder dicernir finalmente la coincidencia final se utilizó 3 grupos humanos, conformados por 2 personas por equipo, para realizar el descarte final de la lista de medicamentos.

3. **Carga de GTIN al catálogo de medicamentos**: Luego de que se concluye el proceso de verificación manual, la data resultante se carga a la tabla **OPPM.Catalogo_Medicamentos** en el campo **GTIN**

In [ ]:
-- INSERCIÓN DE LA INFORMACIÓN AL CATÁLOGO DE MEDICAMENTOS
UPDATE CM
SET CM.GTIN = OG.GTIN
FROM OPPM.Catalogo_Medicamentos CM
INNER JOIN Temp.OPPM_GTIN OG
ON CM.Cod_Prod = OG.Cod_Prod

4. **Identificación de lista de productos a trabajar en Kairos**: Basado en los productos identificados en el proceso de Homologación GTIN, se extrae la relación de productos como lista prioritaria para la homologación Kairos

In [ ]:
--RELACIÓN DE PRODUCTOS DEL OBSERVATORIO Y SUS ATRIBUTOS QUE PUEDEN TENER RELACIÓN CON EL CATÁLOGO KAIROS
SELECT CM.Cod_Prod, CM.Prin_Activo, CM.Nom_Prod, CM.Concent, CM.Fracciones, CM.nom_form_farm_simp, CM.Laboratory, CM.Presentac
FROM OPPM.Catalogo_Medicamentos CM
INNER JOIN Catalogo.Consumo_TEDEF CT
ON CM.GTIN = CT.GTIN

5. **Generación del Algoritmo de búsqueda Kairos**: este algoritmo nos permite identificar los códigos prioritarios y los atributos que tengan similutud a los datos del observatorio, por ejemplo: **Nombre, fracciones, etc**. Para dicho proceso se utilizó la información del observatorio luego de la homologación con GTIN, **OPPM.Catalogo_Medicamentos**. El algoritmo está conformado por dos metodologías de búsqueda, la primera basada en búsqueda mediante un modelo de **machine learning** (Freetextable) y la segunda basada en **coincidencias** en el contenido de palabras (Containstable). El algoritmo tiene como objetivo comprar ambos catálogos (OPPM y Kairos) para dar como resultado las dos mejores posibilidades de coincidencia.

In [ ]:
-- CATÁLOGO DE KAIROS HISTÓRICO PROVISTO POR APEPS (ÜLTIMA ACTUALIZACIÓN 21 DE JULIO)
SELECT *
FROM Kairos.Catalogo_Medicamentos

In [ ]:
-- CATÁLOGO DE MEDICAMENTOS DEL OBSERVATORIO
SELECT *
FROM Kairos.Catalogo_Medicamentos

In [ ]:
--ALGORITMO DE BÚSQUEDA KAIROS: BUSCA Y ALMACENA LA INFORMACIÓN EN UNA TABLA QUE SERÁ UTILIZADA POSTERIORMENTE
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE PROCEDURE [dbo].[sp_OPPM_KAIROS_V6] @INICIO INT
AS
DECLARE @Codigo         NVARCHAR(30),
        @Prin_Activo    NVARCHAR(50),
        @Nombre         NVARCHAR(50),
        @Lab            NVARCHAR(50),
        @Concent        NVARCHAR(50),
        @Fraccion       NVARCHAR(50),
        @Present        NVARCHAR(50),
        @Form_Farm      NVARCHAR(50),
        @PK             INT = @INICIO,
        @MAX            INT,
        @y              NVARCHAR(255),
        @x              NVARCHAR(255),
        @PESO1          NVARCHAR(50),
        @PESO2          NVARCHAR(50),
        @PESO3          NVARCHAR(50),
        @PESO4          NVARCHAR(50)


SET @MAX = (SELECT MAX(pk_Catalogo_producto) FROM Productos)
SET @PESO1 = 'WEIGHT(0.4)'
SET @PESO2 = 'WEIGHT(0.8)'
SET @PESO3 = 'WEIGHT(1)'
SET @PESO4 = 'WEIGHT(0.6)'

WHILE @PK <= @MAX
BEGIN

SET @Codigo = (SELECT Cod_Prod FROM Productos WHERE PK_Catalogo_Producto = @PK)
SET @Prin_Activo = (SELECT Prin_Activo FROM Productos WHERE PK_Catalogo_Producto = @PK)
SET @Nombre = (SELECT Nom_Prod FROM Productos WHERE PK_Catalogo_Producto = @PK)
SET @Lab = (SELECT Laboratory FROM Productos INNER JOIN Productos_Laboratorio ON Productos.Cod_Prod = Productos_Laboratorio.Cod_Prod WHERE PK_Catalogo_Producto = @PK)
SET @Concent = (SELECT Concent FROM Productos WHERE PK_Catalogo_Producto = @PK)
SET @Fraccion = (SELECT convert(NVARCHAR(50),Fracciones) FROM Productos WHERE PK_Catalogo_Producto = @PK)
SET @Present = (SELECT Presentac FROM Productos WHERE PK_Catalogo_Producto = @PK)
SET @Form_Farm = (SELECT M.EQUIVALENTE FROM Productos AS P LEFT JOIN MAESTRO_FORMA_FARM AS M ON p.Nom_Form_Farm_Simplif = M.Nom_Form_Farm_Simplif WHERE PK_Catalogo_Producto = @PK)

SET @y = 'ISABOUT(' + '"*'+ ISNULL(@Prin_Activo,'') + '*"' + ' ' + @PESO2 + ', ' + '"*'  + ISNULL(@Nombre,'') + '*"'  + ' ' + @PESO3 + ', ' + '"*'  + ISNULL(@Lab,'') + '*"'  + ' ' + @PESO3 + ', ' + '"*'  + ISNULL(@Concent,'') + '*"'  + ' ' + @PESO4 + ', ' + '"'  + ISNULL(@Form_Farm,'') + '*"'  + ' ' + @PESO4 + ', '+ '"'  + ISNULL(@Fraccion,'') + '"'  + ' ' + @PESO1  + ')';
SET @x = ISNULL(@Prin_Activo,'') + ' ' + ISNULL(@Nombre,'') + ' ' + ISNULL(@Lab,'') + ' ' + ISNULL(@Concent,'') + ' ' + ISNULL(@Form_Farm,'') + ' ' + ISNULL(@Fraccion,'');

-- PRINT (@y)
-- PRINT (@x)

WITH x AS
(
    SELECT top 2 @PK [PK], KEY_TBL.[RANK], @Codigo [OPPM_CODIGO], Kairos.CODIGO [Kairos_CODIGO], Kairos.PRD_DESC_COMPLETO, @Prin_Activo [OPPM_PRIN_ACT], @Nombre [OPPM_NOM], @Lab [OPPM_LAB], @Concent [OPPM_CONCENT], @Fraccion [OPPM_FRAC], @Form_Farm [Forma_Farm], @Present [OPPM_PRES], 'C' [TB]
    FROM Kairos
    INNER JOIN CONTAINSTABLE(Kairos, PRD_DESC_COMPLETO,  
                                        @y) AS KEY_TBL
                    ON Codigo = KEY_TBL.[KEY]
    ORDER BY KEY_TBL.[RANK] DESC
), y AS
(
    SELECT top 2 @PK [PK], KEY_TBL.[RANK], @Codigo [OPPM_CODIGO], Kairos.CODIGO [Kairos_CODIGO], Kairos.PRD_DESC_COMPLETO, @Prin_Activo [OPPM_PRIN_ACT], @Nombre [OPPM_NOM], @Lab [OPPM_LAB], @Concent [OPPM_CONCENT], @Fraccion [OPPM_FRAC], @Form_Farm [Forma_Farm], @Present [OPPM_PRES], 'F' [TB]
    FROM Kairos
    INNER JOIN FREETEXTTABLE(Kairos, PRD_DESC_COMPLETO,  
                                        @x) AS KEY_TBL
                    ON Codigo = KEY_TBL.[KEY]
    ORDER BY KEY_TBL.[RANK] DESC
), xy AS
(
    SELECT *
    FROM y
    UNION ALL
    SELECT *
    FROM x
)
INSERT INTO OPPM_KAIROS_V2
SELECT TOP 2 *
FROM xy
ORDER BY [RANK] DESC

SET @PK = @PK + 1

END
GO


6. **Revisión por pares en 3 grupos**: Una vez ejecutado el algoritmo, se obtiene como resultado una lista de **12 mil registros** a comprar, con **top 2** de medicamentos con mayores coincidencias para cada producto del catálogo del observatorio. Para poder dicernir finalmente la coincidencia final se utilizó 3 grupos humanos, conformados por 2 personas por equipo, para realizar el descarte final de la lista de medicamentos.

7. **Carga de Kairos al catálogo de medicamentos**: Luego de que se concluye el proceso de verificación manual, la data resultante se carga a la tabla **OPPM.Catalogo\_Medicamentos** en el campo **Kairos**

In [ ]:
-- INSERCIÓN DE LA INFORMACIÓN AL CATÁLOGO DE MEDICAMENTOS
UPDATE CM
SET CM.Kairos = OK.Kairos
FROM OPPM.Catalogo_Medicamentos CM
INNER JOIN Temp.OPPM_Kairos OK
ON CM.Cod_Prod = OK.Cod_Prod

**NOTA**: En base a la nueva información provista por SuSalud y evidencia de nuevas casuísticas en el proceso de homologación se está desarrollando una 7ma versión del algortimo de búsqueda, donde se espera que mejore la precisión de las coincidencias del algoritmo.